# Kafka

## Kafka相关概念

Kakfa定位为一个分布式流式处理平台，高吞吐、可持久化、可水平扩展、支持流式数据处理。kafka可以被用作：

* 消息系统，实现系统解耦、冗余存储、流量削峰，kafka还提供了大多数消息系统难以实现的**消息顺序性保障**和**消息回溯**功能
* 存储系统，kafka把消息持久化到磁盘
* 流式处理平台

### kafka技术架构

![](https://bj.bcebos.com/ipic/Kafka体系架构.jpg)

* Producer生产者，发送消息的一方，负责把消息发送到kafka中
* Consumer消费者，接收消息的一方，消费者连接到kafka上接收消息，并处理相应的业务逻辑
* Broker服务代理节点，Broker可以看做一个Kakfa服务节点或者实例。
* ZooKeeper，管理集群的元数据

### 主题和分区

主题是一个逻辑上的概念，它还可以细分多个分区，一个分区只属于单个主题。同一主题下的不同分区包含的消息式不同的，分区在存储层面可以看作一个可追加的日志（Log）文件，消息在被追加到分区日志文件时候都会分配一个特定的偏移量（offset）。offset是消息在分区中的唯一标识，**kafka通过offset保证分区内的顺序性，不过offset并不跨分区，kafka保证的式分区有序而不是主题有序**。

### 副本

kafka为分区引入了多副本（Replic）机制，通过增加副本数量可以提升容灾能力，副本之间式"一主多从"的关系，其中leader副本负责处理读写请求，follower副本只负责与leader副本的消息同步。副本处于不同的broker中，当leader副本发生故障时，从follower副本重新选举leader副本对外提供服务。Kafka通过多副本机制实现了故障的自动转移，当kafka某个broker失效时仍然能保证服务可用。

分区中所有的副本统称为AR（Assigned Replicas），所有与leader副本保持一定程度同步的副本组成ISR（In-Sync Replicas），与leader副本滞后过多的副本组成OSR（Out-Of-Sync Replicas）。 `AR = ISR + OSR`

leader副本故障时，只有ISR的副本才有资格被选举成leader副本。

![kafka分区偏移量说明](https://bj.bcebos.com/ipic/Kafka分区中偏移量说明.jpg)

ISR与HW和LEO有紧密的关系。HW是High Watermark的缩写，俗称高水位，它标识了一个特殊的偏移量（offset），消费者只能拉取这个offset之前的消息。LEO是Log End Offset的缩写，它标识当前日志文件中下一条待写入消息的偏移offset，LEFO的大小等于当前日志分区中最后一条消息的offset加1。分区ISR集合中的每个副本都会维护自己的LEO，ISR集合中最小LEO为分区的HW，对于消费者而言只能消费HW之前的消息。

kafka的复制机制既不是完全的同步复制，也不是单纯的异步复制。同步复制要求所有的follower副本都复制完，这条消息才会被确认提交，这种方式极大的影响了性能。异步复制，follower副本异步地从leader副本中复制数据，数据只被leader副本写入就被认为已经成功提交。在这种情况下，如果follower副本复制都落后于leader副本，突然leader副本宕机，则会造成数据丢失。kafka使用这种ISR的方式有效地权衡了数据可靠性和性能之间的关系。

## 生产者

### 重要的生产者参数

* `acks`这个参数用来指定分区中必须要有多少个副本收到这条消息，之后生产者才会认为这条消息是成功写入的。
    * `acks=1`，默认值为1，生产者发送消息之后，只要分区中的leader副本写入消息，那么就会收到服务端的成功响应
    * `acks=0`，生产者发送消息之后，不需要等待任何服务端的响应
    * `acks=-1`或者`acks=all`，生产者在发送消息成功后，需要等待ISR中的所有副本都成功写入消息后，才能收到服务端的响应

## 消费者

### 消费者和消费者组

消费者（Consumer）负责订阅kafka中的主题（Topic），并且从订阅的主题上拉取消息。在kafka中，还有消费者组（Consumer Group）的概念，每个消费者都有一个对应的消费者组，当消费者发布到主题后，只会被投递到它的每个消费者组中的一个消费者。

![](https://bj.bcebos.com/ipic/Kafka消费者与消费者组.jpg)

消费者组是一个逻辑上的概念，它将旗下的消费者归为一类，每个消费者只隶属于一个消费者组。

### 客户端开发

一个正常的消费逻辑包括以下几个步骤：

1. 配置客户端参数，并创建消费者实例
2. 订阅主题
3. 拉取消息并消费
4. 提交消费位移
5. 关闭消费者实例

### 位移提交

消费者每次调用`poll()`方法时，它返回的是还没有消费过的消息。要做到这一点就必须记录上次消费的消费位移，并且这个消费位移要持久化保存，而不是保存在内存中，否则消费者重启之后就无法知道之前的消费位移。当有新的消费者加入时，必然会有再均衡的操作，对于一个分区而言，它可能在再均衡操作之后分配新的消费者，如果不持久化保存消费位移，那么新的消费者就无法知道之前的消费位移。

#### 重复消费和消息丢失

* 重复消费，消息被消费，位移未提交
* 消息丢失，位移已提交，消息未消费

kafka默认的位移提交方式为自动提交，这个由消费者的客户端参数`enable.auto.commit`配置，默认为`true`。消费者每隔5秒会拉取分区中最大的消费位移进行提交，自动提交位移的动作是再`poll()`方法的逻辑里实现的。**在每次向服务器发起拉取请求之前，会检查是否有新的位移需要提交，如果可以，会提交上一次轮询的位移。**

### 再均衡

再均衡是指分区的所属权从一个消费者转移到另一消费者的行为，它为消费者具备高可用和伸缩性提供了保障，是我们可以方便安全的删除消费者组内的消费者或者往消费者组内添加消费者。**再均衡期间，消费组内的消费者是无法读取消息的。也就是说，再均衡期间，消费者变得不可用。** 另外，当一个分区被重新分配给另一个消费者时，消费者的当前状态也会消失。比如消费者消费完某一个分区的一部分消息时还没有来得及提交消费位移就发生了再均衡操作，之后这个分区又分配给消费者组内另外一个消费者，原来被消费完的消息又被重新消费了一遍，也就是发生了**重复消费**。

## kafka事务

### 消息传输保障

消息传输保障有3个层级：

* `at most once`: 至多一次，消息可能会丢失，但绝对不会重复传输
* `at least once`: 最少一次，消息绝对不会丢失，但可能会重复传输
* `exactly once`: 恰好一次，消息肯定会被传输，且只传输1次

kafka从`0.11.0.0`引入了幂等和事务两个特性，以此实现EOS(Exactly Once Sematics)。

### 幂等

幂等是对接口的多次调用和一次调用产生的结果是一致的。kafka生产者再进行重试的时候可能会重复写入消息，而使用kafka的幂等性功能后可以避免这种情况。

为了实现生产者的幂等性，kafka引入了producer id（简称PID）和sequence number（序列号）。

每个新的生产者实例在初始化的时候都会分配一个PID，这个PID对用户完全是不可见的。

broker端会在内存中为每堆<PID, Partition>维护一个序列号SN_old，新消息的序列号SN_new

* 新消息序列号SN_new比broker维护的序列号SN_old大于1，broker接收该消息
* SN_new小于SN_old+1，说明消息重复写入，该消息被丢弃
* SN_new大于SN_old+1，说明中间有消息未写入，出现了乱序，broker会抛出`OutOfOrderSequenceException`


## FAQ